<a href="https://colab.research.google.com/github/arungovindm/seq2seq/blob/master/seq2seq_wAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Pre-processing parameters
</br>seq length threshold</br>count threshold


##Hyperparamters of the model
</br>batch_size
</br>embedding dimensions
</br>LSTM units

##Install and import dependencies

In [1]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.1-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision

    100% |████████████████████████████████| 483.0MB 33.3MB/s 
fastai 1.0.50.post1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.50.post1 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
  Found existing installation: torch 1.0.1.post2
    Uninstalling torch-1.0.1.post2:
      Successfully uninstalled torch-1.0.1.post2


In [2]:
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import unicodedata
import re
import time
import keras
import string

print(torch.__version__)

Using TensorFlow backend.


0.4.1


##Load data

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
en = open('/content/drive/My Drive/nmt/europarl-v7.de-en.en', encoding='UTF-8').read().strip().split('\n')
de = open('/content/drive/My Drive/nmt/europarl-v7.de-en.de', encoding='UTF-8').read().strip().split('\n')

##Building vocabulary

###Set vocabulary Hyper-parameters

In [0]:
sent_threshold=15
count_threshold=15

In [0]:
de_m=[]
en_m=[]
for i in range(len(de)):
  if len(de[i].split(' '))< sent_threshold+1 and len(en[i].split(' '))< sent_threshold+1:
    en_m.append(en[i])
    de_m.append(de[i])

###Tokenize

In [0]:
from keras.preprocessing.text import Tokenizer
t= Tokenizer(num_words=30000,lower=True,oov_token='<UNK>')
t.fit_on_texts(en_m)

t_d= Tokenizer(num_words=30000,lower=True,oov_token='<UNK>')
t_d.fit_on_texts(de_m)

In [0]:
t2=Tokenizer()
t_d2=Tokenizer()
t2.word_counts['<UNK>']=0
for word,counts in t.word_counts.items():
  if counts < count_threshold:
    t2.word_index[word]=3
    t2.word_counts['<UNK>']=t2.word_counts['<UNK>']+counts
    
  else:
    t2.word_index[word]=t.word_index[word]+2
    t2.word_counts[word]=t.word_counts[word]
    
t_d2.word_counts['<UNK>']=0
for word,counts in t_d.word_counts.items():
  if counts < count_threshold:
    t_d2.word_index[word]=3
    t_d2.word_counts['<UNK>']=t_d2.word_counts['<UNK>']+counts
    
  else:
    t_d2.word_index[word]=t_d.word_index[word]+2
    t_d2.word_counts[word]=t_d.word_counts[word]

In [0]:
t2.word_index['<pad>']=0
t2.word_index['<start>']=1
t2.word_index['<end>']=2

t_d2.word_index['<pad>']=0
t_d2.word_index['<start>']=1
t_d2.word_index['<end>']=2

##Functions to get padded, indexed sequence

In [0]:
def index_batch(t,batch,flag): #batch will be a list of size [batchsize] with each item being a sentence
  for i in range(len(batch)):
    batch[i]=batch[i].split(' ')
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table).lower() for w in batch[i]]
    for j in range(len(batch[i])):
      if stripped[j]  not in t.word_index:
        batch[i][j]=3
      else:
        batch[i][j]=t.word_index[stripped[j].lower()]
    if flag=='decoder':
      start_token=np.array(1)
      stop_token=np.array(2)
      batch[i]=np.hstack([start_token,batch[i],stop_token])

  return batch

In [0]:
def get_batch(text,t,flag='encoder'):  #returns the padded indexed np.array of given batch number
  batch=index_batch(t,text,flag)
  batch=keras.preprocessing.sequence.pad_sequences(batch,dtype=int,padding='post',truncating='post',value=0)
  return batch

##Parameters

In [0]:
global batch_size, data_size,embedding_dims,units
learning_rate = 0.001
BUFFER_SIZE = len(en_m)
batch_size=32
N_BATCH = BUFFER_SIZE//batch_size
units=128
data_size = len(en_m)
en_vocab_size=len(t2.word_counts)+3 #same as below
de_vocab_size=len(t_d2.word_counts)+3 #adding start stop and pad tokens
embedding_dims=150

##Create dataset

In [0]:
from torch.utils.data import Dataset, DataLoader

In [0]:
class MyData(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y
        # TODO: convert this into torch code is possible
        self.length = [ np.sum(1 - np.equal(x, 0)) for x in X]
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        return x,y,x_len
    
    def __len__(self):
        return len(self.data)

In [0]:
input_tensor = get_batch(en_m,t2,'decoder')
target_tensor = get_batch(de_m,t_d2,'decoder')

In [0]:
train_dataset = MyData(input_tensor, target_tensor)
dataset = DataLoader(train_dataset, batch_size, 
                      drop_last=True,
                      shuffle=True)

##Encoder class

In [0]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.LSTM(self.embedding_dim, self.enc_units)
        
    def forward(self, x, lens, device):
        x=x.transpose(0,1)
        x = self.embedding(x) 
        x = x.permute(1,0,2)
        x = pack_padded_sequence(x, lens) # unpad
    
        self.hidden_state = self.initialize_hidden_state(device)
        self.cell_state = self.initialize_hidden_state(device)
        
        output, (self.hidden_state,self.cell_state) = self.gru(x, (self.hidden_state,self.cell_state)) # gru returns hidden state of all timesteps as well as hidden state at last timestep
        
        # pad the sequence to the max length in the batch
        output, _ = pad_packed_sequence(output)
        
        return output, (self.hidden_state, self.cell_state)

    def initialize_hidden_state(self, device):
        return torch.zeros((1, self.batch_sz, self.enc_units)).to(device)

In [0]:
### sort batch function to be able to use with pad_packed_sequence
def sort_batch(X, y, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    y = y[indx]
    return X.transpose(0,1), y, lengths # transpose (batch x seq) to (seq x batch)

##Decoder class

In [0]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dec_units, enc_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.lstm = nn.LSTM(self.embedding_dim + self.enc_units, 
                          self.dec_units,
                          batch_first=True)
        self.fc = nn.Linear(self.enc_units, self.vocab_size)
        
        self.W1 = nn.Linear(self.enc_units, self.dec_units)
        self.W2 = nn.Linear(self.enc_units, self.dec_units)
        self.V = nn.Linear(self.enc_units, 1)
    
    def forward(self, x, hidden,cell, enc_output):
        enc_output = enc_output.permute(1,0,2)
        
        hidden_with_time_axis = hidden.permute(1, 0, 2)
        
        score = torch.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        
        attention_weights = torch.softmax(self.V(score), dim=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector, dim=1)
        
        x = self.embedding(x)
        
        x = torch.cat((context_vector.unsqueeze(1), x), -1)
        
        # passing the concatenated vector to the LSTM
        # output: (batch_size, 1, hidden_size)
        output, (hstate,cstate) = self.lstm(x)
        
        
        # output shape == (batch_size * 1, hidden_size)
        output =  output.view(-1, output.size(2))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, (hstate,cstate) ,attention_weights
    
    def initialize_hidden_state(self):
        return torch.zeros((2,1, self.batch_sz, self.dec_units))

In [0]:
#vanilla seq2seq
class vanillaDecoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dec_units, enc_units, batch_sz):
        super(vanillaDecoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.LSTM(self.embedding_dim, 
                          self.dec_units,
                          batch_first=True)
        self.fc = nn.Linear(self.enc_units, self.vocab_size)
        
    
    def forward(self, x, hidden,cell, enc_output):
        enc_output = enc_output.permute(1,0,2)
        
        hidden_with_time_axis = hidden.permute(1, 0, 2)
        
        x = self.embedding(x)
        
        output, (hstate,cstate) = self.gru(x)
        
        
        output =  output.view(-1, output.size(2))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, (hstate,cstate)     
    def initialize_hidden_state(self):
        return torch.zeros((2,1, self.batch_sz, self.dec_units))

In [0]:
#encoder_outputs shape ([17, 64, 128])
#decoder input ([64, 1])
class Multiplicative_decoder(nn.Module):
  def __init__(self, vocab_size,embedding_dim, enc_units, dec_units, batch_sz):
    super(Multiplicative_decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.enc_units = enc_units
    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
    self.lstm=nn.LSTM(self.embedding_dim,self.dec_units,batch_first=True)
    self.pred_layer=nn.Linear(self.enc_units + self.dec_units, self.vocab_size)
    
    
    #attention part
    k=self.enc_units**(-0.5)
    a=torch.FloatTensor(self.enc_units,self.dec_units).uniform_(-1,1)
    self.W = nn.Parameter(a)
    
    
  def forward(self, x, hstate, cstate, enc_output):
    k=self.enc_units**(-0.5)

    x = self.embedding(x)
    s, (hstate, cstate) = self.lstm(x)
    
    #enc_outputs are seq x batch_size x  enc_units
    sequence_length, batch_size, self.enc_units = enc_output.shape
    enc_output=enc_output.permute(1,0,2)
    
    attention = torch.matmul(torch.matmul(enc_output,self.W) ,hstate.permute(1,2,0))
    attention_weights = torch.softmax(attention,dim=1)
    #print(attention_weights)

    enc_output=enc_output.permute(1,0,2)
    
    attention_weights=attention_weights.permute(1,0,2)

    context_vector = attention_weights * enc_output
    context_vector = torch.sum(context_vector, dim=0)
    pred_vec = torch.cat((context_vector.unsqueeze(2),hstate.permute(1,2,0)),1).squeeze()
    
    output = self.pred_layer(pred_vec)
    
    return output, (hstate,cstate),attention_weights
    

##Define loss function

In [0]:
criterion = nn.CrossEntropyLoss()

def loss_function(real, pred):
    """ Only consider non-zero inputs in the loss; mask needed """
    loss_ = criterion(pred, real) 
    return torch.mean(loss_)

In [0]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

encoder = Encoder(en_vocab_size, embedding_dims, units, batch_size)
decoder = Multiplicative_decoder(de_vocab_size, embedding_dims, units, units, batch_size)

encoder.to(device)
decoder.to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), 
                       lr=0.001)

##Train

In [0]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    encoder.train()
    decoder.train()
    
    total_loss = 0
    
    for (batch, (inp, targ, inp_len)) in enumerate(dataset):
        loss = 0
        
        xs, ys, lens = sort_batch(inp, targ, inp_len)
        
#         print("Output: ", targ.shape)
        enc_output, (enc_hidden,enc_cell) = encoder(xs.to(device), lens, device)
        dec_hidden = enc_hidden
        dec_cell = enc_cell
        #print(enc_output.size())
        
        
        # use teacher forcing - feeding the target as the next input (via dec_input)
        dec_input = torch.tensor([[t_d2.word_index['<start>']]] * batch_size)
        #print(dec_input.size())
        # run code below for every timestep in the ys batch
        for t in range(1, ys.size(1)):
            predictions, (dec_hidden,dec_cell),att_weights = decoder(dec_input.to(device),
                                                            dec_hidden.to(device),
                                                            dec_cell.to(device),
                                                            enc_output.to(device))
            #print('prediction shape',predictions.size())
            loss += loss_function(ys[:, t].to(device), predictions.to(device))
            #loss += loss_
            dec_input = ys[:, t].unsqueeze(1)
            
        
        batch_loss = (loss / int(ys.size(1)))
        total_loss += batch_loss
        
        optimizer.zero_grad()
        
        loss.backward()

        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
#         if batch % 100 == 0:
#           print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
#                                                            batch,
#                                                            batch_loss.detach().item()))
        
        
    ### TODO: Save checkpoint for model
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    if epoch ==4:
      torch.save(encoder.state_dict(),'/content/drive/My Drive/nmt/mult/enc_64')
      torch.save(decoder.state_dict(),'/content/drive/My Drive/nmt/mult/dec_64')
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    
    
    
    

###Saving the model

##Saving learned parameters

In [0]:
torch.save(encoder.state_dict(),'/content/drive/My Drive/nmt/hstate_50_dim/enc_mult')

In [0]:
torch.save(decoder.state_dict(),'/content/drive/My Drive/nmt/hstate_50_dim/dec_mult')

##Loading learned parameters

In [194]:
del device
device=torch.device("cpu")
add_dec = Decoder(de_vocab_size, embedding_dims, units, units, 1)
add_dec.load_state_dict(torch.load('/content/drive/My Drive/nmt/additive_embedding_150_bsize64_s15_fr15/dec'))
add_enc = Encoder(en_vocab_size, embedding_dims, units, 1)
add_enc.load_state_dict(torch.load('/content/drive/My Drive/nmt/additive_embedding_150_bsize64_s15_fr15/enc'))
add_dec.eval()
add_enc.eval()

Encoder(
  (embedding): Embedding(9714, 150)
  (gru): LSTM(150, 128)
)

In [0]:
index_dict=dict()
for word, index in t_d2.word_index.items():
  index_dict[index]=word
  
en_index_dict=dict()
for word, index in t2.word_index.items():
  en_index_dict[index]=word
  
  
index_dict[3]='<UNK>'

##Test set

In [0]:
test_en = open('/content/drive/My Drive/nmt/test_set/newssyscomb2009.en', encoding='UTF-8').read().strip().split('\n')
test_de = open('/content/drive/My Drive/nmt/test_set/newssyscomb2009.de', encoding='UTF-8').read().strip().split('\n')

In [0]:
def tokenize(batch):
  for i in range(len(batch)):
      batch[i]=batch[i].split(' ')
      table = str.maketrans('', '', string.punctuation)
      batch[i] = [w.translate(table).lower() for w in batch[i]]
  return batch

In [0]:
## sort batch function to be able to use with pad_packed_sequence
def sort_testbatch(X, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    return X.transpose(0,1), lengths # transpose (batch x seq) to (seq x batch)

In [0]:
global attn
def translate(add_enc,add_dec,test,length):
  sent1=""
  # sent2=""
  output, (hstate,cstate)=add_enc(test.to(device),length.to(device),device)
  dec_input = torch.tensor([[t_d2.word_index['<start>']]] * 1)
  dec_hidden=hstate
  dec_cell=cstate
  word1=""
  # word2=""

  predictions, (dec_hidden,dec_cell),attn = add_dec(dec_input.to(device),
                                                                dec_hidden.to(device),
                                                                dec_cell.to(device),
                                                                output.to(device))
  print(attn)
  #predictions=predictions.unsqueeze(0)
  t=0
  while word1 !='<end>' and t<17:
    _, indices = predictions.max(1)
    
#     print(indices.shape)
    word1= index_dict[int(indices[0])]
  #   word2= index_dict[int(indices[1])]
    sent1 = sent1+" "+ word1
  #   sent2 = sent2+" "+ word2
    dec_input = torch.tensor([[int(indices[0])]])
    predictions, (dec_hidden,dec_cell),attn = add_dec(dec_input.to(device),
                                                                dec_hidden.to(device),
                                                                dec_cell.to(device),
                                                                output.to(device))
    #predictions=predictions.unsqueeze(0)
    t=t+1
  return sent1

In [0]:
de_t=[]
en_t=[]
tester_bitch=[]
tester_bitch2=[]
for i in range(len(test_de)):
  if len(test_de[i].split(' '))< sent_threshold+1 and len(test_en[i].split(' '))< sent_threshold+1:
    tester_bitch.append(test_en[i])
    tester_bitch2.append(test_de[i])
    #print(test_en[i])
    en_t.append(test_en[i])
    de_t.append(test_de[i])
tester_bitch=tokenize(tester_bitch)
long = []
for i in tester_bitch:
  long.append(len(i))


##BLEU Calculation

In [0]:
long=torch.tensor(long)
long, indx = long.sort(dim=0, descending=True)
for i in range(len(long)):
  tester_bitch[i] = tester_bitch[indx[i]]

In [0]:
test_input_tensor = get_batch(en_t,t2,'decoder')
test_target_tensor = get_batch(de_t,t_d2,'decoder')

In [0]:
length = [ np.sum(1 - np.equal(x, 0)) for x in test_input_tensor]
length=torch.tensor(length,dtype=torch.int32)
test_ins,lengths = sort_testbatch(test_input_tensor,length)
test_outs,leng = sort_testbatch(test_target_tensor,length)
test_ins=torch.tensor(test_ins)
sentence=[]
for i in range(len(tester_bitch)):
  test=test_ins[i,:].unsqueeze(1)
  l=torch.tensor([lengths[i].tolist()])
  #print(l)
  sentence.append(translate(add_enc,add_dec,test,l))

In [0]:
source=[]
test_outs=test_outs.transpose(0,1)
for i in range(len(test_outs)):
  sent=""
  for j in range(len(test_outs[i])):
    
    if int(test_outs[i][j]) >2: 
      sent=sent+index_dict[int(test_outs[i][j])]+' '
    
  source.append(sent)

In [0]:
for i in range(len(source)):
  source[i]=source[i].split(' ')
  sentence[i]=sentence[i].split(' ')
  source[i].append('<end>')
  

In [0]:
for i in range(len(source)):
  source[i]= list(filter( lambda x: x not in ['<end>','','<start>','<pad>'],source[i]))
  sentence[i]= list(filter( lambda x: x not in ['<end>','','<start>','<pad>'],sentence[i]))

In [0]:
ref = []
for sent in source:
  ref.append([sent])

In [231]:
from nltk.translate.bleu_score import corpus_bleu
corpus_bleu(ref,sentence)

0.2084896067711646

seq2seq 1.180844816106659</br>
Additive 128 enc units---> 8.525364965063201</br>
Additive 64 enc units ----> 7.966707348359717</br>
Multiplicative 3.8322325669944445  not --->0.012138092763158777</br>
Scaled dot 3.797474380138463 not ---->0.031393753345098786</br>

  